## Business Understanding
Predicting the type  

## Data Understanding

In [25]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plot


In [26]:
#loading the data
df=pd.read_excel('my_data.xlsx')
df.head()

,customer,location,soap_type,quantity,age,payment_type,Unnamed: 6,Points,Explanation/Questions
0,1,rural,single,3.0,34.0,check,NaN,NaN,Type of soap is your outcome variable
1,2,rural,single,5.0,43.0,credit card,NaN,10,Summarize the data
2,3,rural,single,7.0,91.0,credit card,NaN,10,Clean up the data set and document your actions
3,4,rural,single,5.0,91.0,check,NaN,10,Summarize the data after clean up
4,5,rural,twin,3.0,27.0,cash,NaN,10 each,Create at least four sets of visuals to descri...


In [27]:
df.tail()

,customer,location,soap_type,quantity,age,payment_type,Unnamed: 6,Points,Explanation/Questions
4995,4996,urban,liquid,9.0,20.0,check,NaN,NaN,NaN
4996,4997,urban,liquid,2.0,70.0,credit card,NaN,NaN,NaN
4997,4998,urban,liquid,1.0,21.0,check,NaN,NaN,NaN
4998,4999,urban,liquid,8.0,76.0,cash,NaN,NaN,NaN
4999,5000,urban,liquid,9.0,86.0,cash,NaN,NaN,NaN


In [28]:
df.describe()

,customer,quantity,age,Unnamed: 6
count,5000.000000,4500.000000,4500.000000,0.0
mean,2500.500000,5.524000,58.559111,NaN
std,1443.520003,2.847085,23.337637,NaN
min,1.000000,1.000000,18.000000,NaN
25%,1250.750000,3.000000,38.000000,NaN
50%,2500.500000,6.000000,58.000000,NaN
75%,3750.250000,8.000000,79.000000,NaN
max,5000.000000,10.000000,98.000000,NaN


In [29]:
df.dtypes

customer                   int64
location                  object
soap_type                 object
quantity                 float64
age                      float64
payment_type              object
Unnamed: 6               float64
Points                    object
Explanation/Questions     object
dtype: object

### Data Cleaning

In [30]:
## dropping unwanted columns
df.drop(axis=1,columns=['Unnamed: 6','Points','Explanation/Questions'],inplace=True)


In [31]:
df

,customer,location,soap_type,quantity,age,payment_type
0,1,rural,single,3.0,34.0,check
1,2,rural,single,5.0,43.0,credit card
2,3,rural,single,7.0,91.0,credit card
3,4,rural,single,5.0,91.0,check
4,5,rural,twin,3.0,27.0,cash
...,...,...,...,...,...,...
4995,4996,urban,liquid,9.0,20.0,check
4996,4997,urban,liquid,2.0,70.0,credit card
4997,4998,urban,liquid,1.0,21.0,check
4998,4999,urban,liquid,8.0,76.0,cash


df

In [ ]:
# correcting

In [33]:
def data_checking(df1):
    "A function that checks for missing values and duplicates"
    print("Columns and thier mising values")
    missing=df1.isna().sum()
    print(missing)
    
    print('________')
    print('Percentage of missing values')
    perc_missing=df1.isna().sum()/len(df1)*100
    print(perc_missing)
    
    print('____________')
    print('This are the duplicates')
    
    dups=df1.duplicated().sum()
    print(dups)
    
    

In [34]:
data_checking(df)

Columns and thier mising values
customer          0
location          0
soap_type         0
quantity        500
age             500
payment_type    500
dtype: int64
________
Percentage of missing values
customer         0.0
location         0.0
soap_type        0.0
quantity        10.0
age             10.0
payment_type    10.0
dtype: float64
____________
This are the duplicates
0


In [35]:
df.head()

,customer,location,soap_type,quantity,age,payment_type
0,1,rural,single,3.0,34.0,check
1,2,rural,single,5.0,43.0,credit card
2,3,rural,single,7.0,91.0,credit card
3,4,rural,single,5.0,91.0,check
4,5,rural,twin,3.0,27.0,cash


In [36]:
#filling the quantity  missing values
mean_val=df['quantity'].mean()
df['quantity'].fillna(value=mean_val,inplace=True)

#filling the age missing values
age_val=df['age'].mean()
df['age'].fillna(value=age_val,inplace=True)

df['payment_type'].ffill(inplace=True)

In [38]:
df.isna().sum()/len(df)*100

customer        0.0
location        0.0
soap_type       0.0
quantity        0.0
age             0.0
payment_type    0.0
dtype: float64

In [41]:
df = df.astype({"quantity": int, "age": int})

In [42]:
df.dtypes

customer         int64
location        object
soap_type       object
quantity         int32
age              int32
payment_type    object
dtype: object

## Feature engineering

In [44]:
df['quantity'].value_counts()

5     931
7     496
6     474
3     472
8     455
9     452
2     443
1     433
10    428
4     416
Name: quantity, dtype: int64

In [45]:
df['age'].value_counts()

58    564
64     72
88     71
30     71
38     70
     ... 
70     44
18     44
92     42
80     39
61     38
Name: age, Length: 81, dtype: int64

In [46]:
df['payment_type'].value_counts()

credit card    1698
cash           1664
check          1638
Name: payment_type, dtype: int64

## Exploratory data analysis

### Univariate Analysis

In [43]:
df.head()

,customer,location,soap_type,quantity,age,payment_type
0,1,rural,single,3,34,check
1,2,rural,single,5,43,credit card
2,3,rural,single,7,91,credit card
3,4,rural,single,5,91,check
4,5,rural,twin,3,27,cash


In [ ]:
def cou